In [89]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import date
from datetime import datetime
from datetime import timedelta

In [90]:
data = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')
display(data.shape)
data.head()

(9134, 24)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [91]:
cols=[]
for column in data.columns:
    cols.append(column.lower())
data.columns=cols    

In [92]:
# remove spaces in columns names
data.columns = data.columns.str.replace(' ','_')

Find all of the categorical data. Save it in a categorical_df variable.

In [93]:
data_cat = data.select_dtypes(include='object')
data_cat.head()

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


Check for NaN values.

In [94]:
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer           9134 non-null   object
 1   state              9134 non-null   object
 2   response           9134 non-null   object
 3   coverage           9134 non-null   object
 4   education          9134 non-null   object
 5   effective_to_date  9134 non-null   object
 6   employmentstatus   9134 non-null   object
 7   gender             9134 non-null   object
 8   location_code      9134 non-null   object
 9   marital_status     9134 non-null   object
 10  policy_type        9134 non-null   object
 11  policy             9134 non-null   object
 12  renew_offer_type   9134 non-null   object
 13  sales_channel      9134 non-null   object
 14  vehicle_class      9134 non-null   object
 15  vehicle_size       9134 non-null   object
dtypes: object(16)
memory usage: 1.1+ MB


Check all unique values of columns.

In [95]:
data_cat['gender'].unique()

array(['F', 'M'], dtype=object)

In [96]:
def uniquevalue_df(df):
    
    uniquevalues=[]
    
    df2=df.copy()
    for col in df2.columns:
        uniquevalues.append(df2[col].unique())
    return list(zip(df2.columns,uniquevalues))

uniquevalue_df(data_cat)

# i wanted to look nicer and i have tried a lot but i don't know how to make it any better. 


[('customer',
  array(['BU79786', 'QZ44356', 'AI49188', ..., 'TD14365', 'UP19263',
         'Y167826'], dtype=object)),
 ('state',
  array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon'],
        dtype=object)),
 ('response', array(['No', 'Yes'], dtype=object)),
 ('coverage', array(['Basic', 'Extended', 'Premium'], dtype=object)),
 ('education',
  array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
        dtype=object)),
 ('effective_to_date',
  array(['2/24/11', '1/31/11', '2/19/11', '1/20/11', '2/3/11', '1/25/11',
         '1/18/11', '1/26/11', '2/17/11', '2/21/11', '1/6/11', '2/6/11',
         '1/10/11', '1/17/11', '1/5/11', '2/27/11', '1/14/11', '1/21/11',
         '2/5/11', '1/29/11', '2/28/11', '2/12/11', '2/2/11', '2/7/11',
         '1/22/11', '2/13/11', '1/15/11', '1/8/11', '1/11/11', '1/28/11',
         '2/8/11', '2/23/11', '1/2/11', '2/16/11', '1/27/11', '1/23/11',
         '1/9/11', '2/11/11', '2/4/11', '2/1/11', '2/15/11', '2/26/11',
    

Check dtypes. Do they all make sense as categorical data?

In [97]:
data_cat.dtypes

customer             object
state                object
response             object
coverage             object
education            object
effective_to_date    object
employmentstatus     object
gender               object
location_code        object
marital_status       object
policy_type          object
policy               object
renew_offer_type     object
sales_channel        object
vehicle_class        object
vehicle_size         object
dtype: object

with the exception of "effective_to_date" which can  also be used as categorical, depending on what we are doing, they all make sense being categorical. "renew_offer_type" of the nominal categorical, can be readily dummified by dropping "offer" part that comes before the number.  for the sake of the exercise,"effective_to_date" will be changed to datatime format. 

In [98]:
data_cat['effective_to_date'] = pd.to_datetime(data_cat['effective_to_date'], errors='coerce')

In [99]:
data_cat=data_cat.select_dtypes(include='object')



In [100]:
data_cat.dtypes

customer            object
state               object
response            object
coverage            object
education           object
employmentstatus    object
gender              object
location_code       object
marital_status      object
policy_type         object
policy              object
renew_offer_type    object
sales_channel       object
vehicle_class       object
vehicle_size        object
dtype: object

Does any column contain alpha and numeric data? Decide how to clean it.

"renew_offer_type" of the nominal categorical can be readily dummified by dropping "offer" part that comes before the number.

In [101]:
def renew_offer_num(text):
    return int(text.replace("Offer", ''))

def adding_renew_offer_num(df):
    df2=df.copy()
    df2['renew_offer_type']=df2['renew_offer_type'].apply(renew_offer_num)
    return df2

In [102]:
data_cat1=adding_renew_offer_num(data_cat)
display(data_cat1)

,customer,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal Auto,Personal L1,2,Web,Four-Door Car,Medsize
9130,PK87824,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,1,Branch,Four-Door Car,Medsize
9131,TD14365,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate Auto,Corporate L2,1,Branch,Four-Door Car,Medsize
9132,UP19263,California,No,Extended,College,Employed,M,Suburban,Married,Personal Auto,Personal L2,3,Branch,Four-Door Car,Large


Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.

Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?

initially looking at the data from all 3 columns(coverage, policy and policy type) i thought thatincorporates policy type ( personal, corporate and special) with extra granularity summarized in L1, L2 and L3. with detailed studying of the data as shown below, it is a wronge assumptions and the L1,L2 and L3 don't seem to mean anything. IT COULD BE THAT THESE L1, L2 AND L3 MEAN SOMETHING, but i currently the relationship cannot not established.The danger is in the possibility that L1, L2 and L3 relate to any of the categorical columns having 3 classes such as marital status, area code or agent.  However, for the sake of the exercise,  it make sense to drop off policy given the information provdied and drop "auto" from the policy type and keep coverage data. coverage data can be changed to 1,2,3, but that will be left to the one hot encoder to encode along with the remaining categorical data such as agent, area code and type of vehical data columns. 

In [104]:
data_comparison=data_cat1[['coverage','policy_type', 'policy']]
display(data_comparison)

,coverage,policy_type,policy
0,Basic,Corporate Auto,Corporate L3
1,Extended,Personal Auto,Personal L3
2,Premium,Personal Auto,Personal L3
3,Basic,Corporate Auto,Corporate L2
4,Basic,Personal Auto,Personal L1
...,...,...,...
9129,Basic,Personal Auto,Personal L1
9130,Extended,Corporate Auto,Corporate L3
9131,Extended,Corporate Auto,Corporate L2
9132,Extended,Personal Auto,Personal L2


In [105]:
def clean_policy(text):
    if isinstance(text,str):
        return (text[:-4])
    else:
        return text
    
def clean_policy_add(df):
    df2=df.copy()
    df2['policy_type'] = data_cat1['policy_type'].apply(clean_policy)

In [106]:
data_cat1['policy_type'] = data_cat1['policy_type'].apply(clean_policy)
display(data_cat1.head())

,customer,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate,Corporate L3,1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal,Personal L3,3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal,Personal L3,1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate,Corporate L2,1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal,Personal L1,1,Agent,Four-Door Car,Medsize


In [107]:
data_cat2=data_cat1.drop(columns='policy')

In [108]:
display(data_cat2)

,customer,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate,1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal,3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal,1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate,1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal,1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,No,Basic,Bachelor,Employed,M,Urban,Married,Personal,2,Web,Four-Door Car,Medsize
9130,PK87824,California,Yes,Extended,College,Employed,F,Suburban,Divorced,Corporate,1,Branch,Four-Door Car,Medsize
9131,TD14365,California,No,Extended,Bachelor,Unemployed,M,Suburban,Single,Corporate,1,Branch,Four-Door Car,Medsize
9132,UP19263,California,No,Extended,College,Employed,M,Suburban,Married,Personal,3,Branch,Four-Door Car,Large


The END 